In [6]:
from tkinter import *
import numpy as np
import nltk
import re

class TextSummarizer:
    def casefolding(self, sentence):
        return sentence.lower()

    def cleaning(self, sentence):
        return re.sub(r'[^a-z]', ' ', re.sub("’", '', sentence))

    def tokenization(self, sentence):
        return sentence.split()

    def stopword_removal(self, token): 
        stopword = nltk.corpus.stopwords.words('indonesian')
        stopwords = tuple(stopword)
        temp = []
        for i in range(len(token)):
            if token[i] not in stopwords:
                temp.append(token[i])
        return temp

    def sentence_split(self, paragraph):
        return nltk.sent_tokenize(paragraph)

    def word_freq(self, data):
        w = []
        for sentence in data:
            for words in sentence:
                w.append(words)
        bag = list(set(w))
        res = {}
        for word in bag:
            res[word] = w.count(word)
        return res

    def sentence_weight(self, data, wordfreq):
        weights = []
        for words in data:
            temp = 0
            for word in words:
                temp += wordfreq[word]
            weights.append(temp)
        return weights
    
    def data_append(self, sentence_list):
        data = []
        for sentence in sentence_list:
            data.append(self.tokenization(self.cleaning(self.casefolding(sentence))))
        data = list(filter(None, data))
        return data

    def result(self, sentence_list, n, rank):
        result = ''
        sort_list = np.argsort(rank)[::-1][:2] 
        for i in range(n):
            result += '{} '.format(sentence_list[sort_list[i]]) 
            
        return result

    def summarization(self):
        try:
            n = int(entry_n.get())
            paragraph_input = inputtxt.get('1.0', 'end-1c')
            sentence_list = self.sentence_split(paragraph_input)
            data = self.data_append(sentence_list)
            wordfreq = self.word_freq(data)
            rank = self.sentence_weight(data, wordfreq)
            result_text = self.result(sentence_list, n, rank)

            text_output.delete('1.0', END)
            text_output.insert(END, result_text)
        except ValueError:
            text_output.delete('1.0', END)
            text_output.insert(END, "Masukkan angka yang valid untuk jumlah kalimat.")

root = Tk()
root.geometry("600x500")
root.title("Text Summarization")

text_summarizer = TextSummarizer()

label_n = Label(text="Masukkan Jumlah kalimat yang ingin ditampilkan")
entry_n = Entry(root, width=5)
button_summarize = Button(root, height=2, width=20, text="Ringkas", command=text_summarizer.summarization)

label_n.pack()
entry_n.pack()

# Ubah text_input menjadi inputtxt
inputtxt = Text(root, height=10, width=55)
inputtxt.pack()

button_summarize.pack()

text_output = Text(root, height=10, width=55)
text_output.pack()

root.mainloop()